In [1]:
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
FUNC_ANNOT = "pathway"
# FUNC_ANNOT = "COG"
# FUNC_ANNOT = "regulon"

In [3]:
muts_df = pd.read_pickle("./data/SER_df.pkl")
display(len(muts_df),
       )

27

In [4]:
# processing to help a cleaner order of features.
# sorts mutations by position so that mutations to proximal features will be inserted into flow_df next to each other.
muts_df = muts_df.sort_values(by="Position")
muts_df.head()

,index,Details,mutation target annotation,Mut ID,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,oriC,pseudogene,gene,TFBS,promoter,TSS,RBS,cis-regulatory RNA,attenuator terminator,terminator,genetic,genetic features,genomic features,RegulonDB Gene,mutation size,gene count,disrupt,repaired,taxonomy-id,strain-description,base-media,temperature,carbon-source,nitrogen-source,phosphorous-source,sulfur-source,supplement,genetic feature links,operons,operon links,imodulons,imodulon links,COGs,COG links,regulators,regulator links,pathways,pathway links
1363,27,Y356C (TAC→TGC),thrA,NaN,SNP,1403,NaN,A→G,5,SER,50,1,1.0,1,True,"(1403, 1403)",False,False,{ECK120000987},{},{},{},{},{},{},{},True,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",thrA,1,1,False,False,511145,ΔsdaA ΔsdaB ΔtdcG ΔglyA,M9,37 celsius,glucose(4),NH4Cl(1),KH2PO4(3),MgSO4(0.24),L-Serine,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},"[{'name': 'Leu/Ile', 'mutation set count': 3.0...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...",[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,[],{},"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [..."
1345,0,S357R (AGC→CGC),thrA,NaN,SNP,1405,NaN,A→C,3,SER,50,1,1.0,1,True,"(1405, 1405)",False,False,{ECK120000987},{},{},{},{},{},{},{},True,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",thrA,1,1,False,False,511145,ΔsdaA ΔsdaB ΔtdcG ΔglyA,M9,37 celsius,glucose(4),NH4Cl(1),KH2PO4(3),MgSO4(0.24),L-Serine,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},"[{'name': 'Leu/Ile', 'mutation set count': 3.0...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...",[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,[],{},"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [..."
1355,19,S359R (AGT→AGA),thrA,NaN,SNP,1413,NaN,T→A,4,SER,50,1,1.0,1,True,"(1413, 1413)",False,False,{ECK120000987},{},{},{},{},{},{},{},True,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",thrA,1,1,False,False,511145,ΔsdaA ΔsdaB ΔtdcG ΔglyA,M9,37 celsius,glucose(4),NH4Cl(1),KH2PO4(3),MgSO4(0.24),L-Serine,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},"[{'name': 'Leu/Ile', 'mutation set count': 3.0...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...",[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,[],{},"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [..."
1347,1,coding (31‑34/1374 nt),aroP,NaN,MOB,121518,NaN,IS5 (+) +4 bp,3,SER,50,1,1.0,1,True,"(121518, 121518)",False,False,{ECK120000080},{},{},{},{},{},{},{},True,"[{'name': 'aroP', 'RegulonDB ID': 'ECK12000008...","[{'name': 'aroP', 'RegulonDB ID': 'ECK12000008...",aroP,0,1,True,False,511145,ΔsdaA ΔsdaB ΔtdcG ΔglyA,M9,37 celsius,glucose(4),NH4Cl(1),KH2PO4(3),MgSO4(0.24),L-Serine,{'ECK120000080': ['ECK120000080']},"[{'name': 'aroP', 'RegulonDB ID': 'ECK12001447...",{'ECK120014477': ['ECK120000080']},"[{'name': 'GntR/TyrR', 'mutation set count': 1...","{'GntR/TyrR': ['aroP'], 'NagC/TyrR': ['aroP']}",[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Cra', 'mutation set count': 3.0, 's...","{'Cra': ['aroP'], 'TyrR': ['aroP']}",[],{}
1370,28,L26L (TTG→TTA),ybhN,NaN,SNP,822421,NaN,C→T,5,SER,50,1,1.0,1,True,"(822421, 822421)",False,False,{ECK120003005},{},{},{},{},{},{},{},True,"[{'name': 'ybhN', 'RegulonDB ID': 'ECK12000300...","[

In [5]:
# Specific to the AVA publication
temp_df = pd.DataFrame()
for i, r in muts_df.iterrows():
    if "thrA" in r["RegulonDB Gene"]:
        temp_df = temp_df.append(r)
muts_df = temp_df
len(muts_df)

3

In [6]:
col_l = [
    "Mutation Type",
    "disrupt",
    "repaired",
    "genomic features",
    "genetic features",
    "genetic feature links",
    "operons",
    "operon links",
    "regulators",
    "regulator links",
    "COGs",
    "COG links",
    "pathways",
    "pathway links",
    "mutation target annotation",  # for assigning significant flow
    "RegulonDB Gene"  # for debugging
]

annot_df = muts_df.copy()
annot_df = annot_df[col_l]
# annot_df["mutation effect"] = annot_df.apply(lambda mut_row:"repaired" if mut_row["repaired"] else ("disrupt" if mut_row["disrupt"] else "unknown"), axis=1)


display(annot_df.shape, annot_df.head())

(3, 16)

,Mutation Type,disrupt,repaired,genomic features,genetic features,genetic feature links,operons,operon links,regulators,regulator links,COGs,COG links,pathways,pathway links,mutation target annotation,RegulonDB Gene
1363,SNP,0.0,0.0,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},[],{},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [...",thrA,thrA
1345,SNP,0.0,0.0,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},[],{},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [...",thrA,thrA
1355,SNP,0.0,0.0,"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...",{'ECK120000987': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",{'ECK120014725': ['ECK120000987']},[],{},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [...",thrA,thrA


In [7]:
# Just removing all "Function unknown" links since connections to this COG isn't useful.
for i, r in annot_df.iterrows():
    if "Function unknown" in r["COG links"].keys():
        del r["COG links"]["Function unknown"]

In [8]:
mut_type_feat_effect_counts = dict()
genomic_feat_counts = dict()
genetic_feat_counts = dict()
operon_counts = dict()
COG_counts = dict()
regulator_counts = dict()
pathway_counts = dict()
# Using the individual feature columns rather than link columns because this guarantees considering all linked and non-linked features
# for each of these levels, except genomic features.
for _, r in annot_df.iterrows():
    
    for genomic_feat in r["genomic features"]:
        
        # These are usually exclusive of each other per feature.
        # There is a design problem here: A multi-feature mutation could disrupt feature (gene) and repair another. Need to measure the predicted effect per feature rather than mutation to enable this (considering a design bug fix).
        effect = ""
        if r["disrupt"]: effect = "disrupt"
        if r["repaired"]: effect = "repair"
            
        mut_type_feat_effect = genomic_feat["name"] + ' ' + r["Mutation Type"] + ' ' + effect
        if mut_type_feat_effect not in mut_type_feat_effect_counts.keys():
            mut_type_feat_effect_counts[mut_type_feat_effect] = 0
        mut_type_feat_effect_counts[mut_type_feat_effect] += 1
    
    for genomic_feat in r["genomic features"]:
        genomic_feat_annot = genomic_feat["RegulonDB ID"] #+ ' ' + r["Mutation Type"]
        if genomic_feat_annot not in genomic_feat_counts.keys():
            genomic_feat_counts[genomic_feat_annot] = 0
        genomic_feat_counts[genomic_feat_annot] += 1

    for genetic_feat, links in r["genetic feature links"].items():
        if genetic_feat not in genetic_feat_counts.keys():
            genetic_feat_counts[genetic_feat] = 0
        genetic_feat_counts[genetic_feat] += len(links)

    # count the operons according to the amount of links from genes since an operon is only included if its gene is hit.
    for operon, links in r["operon links"].items():
        if operon not in operon_counts.keys():
            operon_counts[operon] = 0
        operon_counts[operon] += len(links)

    # count the COGs according to the amount of links from operons since a COG is only included if linked to a mutated operon (its genes)
    for COG, links in r["COG links"].items():
        if COG not in COG_counts.keys():
            COG_counts[COG] = 0
        COG_counts[COG] += len(links)
        
    for reg, links in r["regulator links"].items():
        if reg not in regulator_counts.keys():
            regulator_counts[reg] = 0
        regulator_counts[reg] += len(links)
        
    for pathway, links in r["pathway links"].items():
        if pathway not in pathway_counts.keys():
            pathway_counts[pathway] = 0
        pathway_counts[pathway] += len(links)

In [9]:
operon_df = pd.read_csv("./data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [10]:
muts_df.head()

,COG links,COGs,Details,Mut ID,Mutation Type,Position,RBS,Reference Seq,RegulonDB Gene,Sequence Change,TFBS,TSS,ale,attenuator terminator,base-media,carbon-source,cis-regulatory RNA,coding,disrupt,exp,flask,gene,gene count,genetic,genetic feature links,genetic features,genomic features,imodulon links,imodulons,index,isolate,mutation size,mutation target annotation,nitrogen-source,operon links,operons,oriC,pathway links,pathways,phosphorous-source,presence,promoter,pseudogene,range,regulator links,regulators,repaired,strain-description,sulfur-source,supplement,taxonomy-id,tech_rep,temperature,terminator
1363,{'Amino acid transport and metabolism': ['ECK1...,[{'name': 'Amino acid transport and metabolism...,Y356C (TAC→TGC),NaN,SNP,1403.0,{},NaN,thrA,A→G,{},{},5.0,{},M9,glucose(4),{},1.0,0.0,SER,50.0,{ECK120000987},1.0,1.0,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...","[{'name': 'Leu/Ile', 'mutation set count': 3.0...",27,1.0,1.0,thrA,NH4Cl(1),{'ECK120014725': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",0.0,"{'Glycine, serine and threonine metabolism': [...","[{'name': 'Glycine, serine and threonine metab...",KH2PO4(3),1.0,{},0.0,"(1403, 1403)",{},[],0.0,ΔsdaA ΔsdaB ΔtdcG ΔglyA,MgSO4(0.24),L-Serine,511145,1.0,37 celsius,{}
1345,{'Amino acid transport and metabolism': ['ECK1...,[{'name': 'Amino acid transport and metabolism...,S357R (AGC→CGC),NaN,SNP,1405.0,{},NaN,thrA,A→C,{},{},3.0,{},M9,glucose(4),{},1.0,0.0,SER,50.0,{ECK120000987},1.0,1.0,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...","[{'name': 'Leu/Ile', 'mutation set count': 3.0...",0,1.0,1.0,thrA,NH4Cl(1),{'ECK120014725': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",0.0,"{'Glycine, serine and threonine metabolism': [...","[{'name': 'Glycine, serine and threonine metab...",KH2PO4(3),1.0,{},0.0,"(1405, 1405)",{},[],0.0,ΔsdaA ΔsdaB ΔtdcG ΔglyA,MgSO4(0.24),L-Serine,511145,1.0,37 celsius,{}
1355,{'Amino acid transport and metabolism': ['ECK1...,[{'name': 'Amino acid transport and metabolism...,S359R (AGT→AGA),NaN,SNP,1413.0,{},NaN,thrA,T→A,{},{},4.0,{},M9,glucose(4),{},1.0,0.0,SER,50.0,{ECK120000987},1.0,1.0,{'ECK120000987': ['ECK120000987']},"[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","[{'name': 'thrA', 'RegulonDB ID': 'ECK12000098...","{'Leu/Ile': ['thrLABC'], 'thrA-KO': ['thrLABC'...","[{'name': 'Leu/Ile', 'mutation set count': 3.0...",19,1.0,1.0,thrA,NH4Cl(1),{'ECK120014725': ['ECK120000987']},"[{'name': 'thrLABC', 'RegulonDB ID': 'ECK12001...",0.0,"{'Glycine, serine and threonine metabolism': [...","[{'name': 'Glycine, serine and threonine metab...",KH2PO4(3),1.0,{},0.0,"(1413, 1413)",{},[],0.0,ΔsdaA ΔsdaB ΔtdcG ΔglyA,MgSO4(0.24),L-Serine,511145,1.0,37 celsius,{}


In [11]:
# operon names can be the same as a gene, which is problemmatic for floweaver
def _adjust_name_for_floweaver(operon):
    return operon + ' '


def _get_feat_d(json, RegulonDB_ID=None, name=None):
    feat_d = dict()
    for d in json:
        if (RegulonDB_ID and d["RegulonDB ID"] == RegulonDB_ID) or (name and d["name"] == name):
            feat_d = d
            break
    return feat_d


# Will probably have to put elsewhere at some point due to annotation work (count, significance)
mut_feat_effects = list()
genomic_feats = list()
genetic_feats = list()
operons = list()
COGs = list()
regulators = list()
pathways = list()


KEY_MUT_PREFIX = ["thr", 'gcv', "rho", "lrp"]
flow_df = pd.DataFrame()
for _, r in annot_df.iterrows():

    signif_flow = "F"
    for k in KEY_MUT_PREFIX:
        if k in r["mutation target annotation"]:
            signif_flow = "T"
            break

    for genomic_feat_d in r["genomic features"]:
        
        effect = ""
        if r["disrupt"]: effect = "disrupt"
        if r["repaired"]: effect = "repair"    
        mut_type_feat_effect = genomic_feat_d["name"] + ' ' + r["Mutation Type"] + ' ' + effect
        mut_type_feat_effect = str(mut_type_feat_effect_counts[mut_type_feat_effect]) + ' ' + mut_type_feat_effect
        mut_feat_effects.append(mut_type_feat_effect)
            
        genomic_feat_annot = ""
        mut_count = genomic_feat_counts[genomic_feat_d["RegulonDB ID"]]
        if genomic_feat_d["significant"] and mut_count > 1: genomic_feat_annot += "*"
        genomic_feat_annot += str(mut_count) + ' '
        genomic_feat_annot += str(genomic_feat_d["name"])
        flow_row = {
            "source": genomic_feat_annot,
            "target": mut_type_feat_effect,
            "type": signif_flow,
            "value": 1
        }
        flow_df = flow_df.append(flow_row, ignore_index=True)
        
    
    
    for genetic_feat_ID, genomic_feat_set in r["genetic feature links"].items():
        genetic_feat_d = _get_feat_d(RegulonDB_ID=genetic_feat_ID, json=r["genetic features"])
        genetic_feat_annot = ""
        mut_count = genetic_feat_counts[genetic_feat_ID]
        if genetic_feat_d["significant"] and mut_count > 1: genetic_feat_annot += "*"
        genetic_feat_annot += str(mut_count) + ' '
        genetic_feat_annot += str(genetic_feat_d["name"])
        genetic_feats.append(genetic_feat_annot)
        for genomic_feat_ID in genomic_feat_set:
            genomic_feat_d = _get_feat_d(RegulonDB_ID=genomic_feat_ID, json=r["genomic features"])
            k = genomic_feat_ID #+ ' ' + r["Mutation Type"]
            genomic_feat_annot = ""
            mut_count = genomic_feat_counts[k]
            if genomic_feat_d["significant"] and mut_count > 1: genomic_feat_annot += "*"
            genomic_feat_annot += str(mut_count) + ' '
            genomic_feat_annot += str(genomic_feat_d["name"]) #+ ' ' + r["Mutation Type"]
#             if r["disrupt"]: genomic_feat_annot += " disrupt"
#             if r["repaired"]: genomic_feat_annot += " repair"
            genomic_feats.append(genomic_feat_annot)
            flow_row = {
                "source": genetic_feat_annot,
                "target": genomic_feat_annot,
                "type": signif_flow,
                "value": 1
            }
            flow_df = flow_df.append(flow_row, ignore_index=True)
    
    
    for operon_id, gen_targs in r["operon links"].items():
        operon_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
        operon_name = _adjust_name_for_floweaver(operon_d["name"])
        operon_annot = ""
        mut_count = operon_counts[operon_id]
        if operon_d["significant"] and mut_count > 1: operon_annot += "*"
        operon_annot += str(mut_count) + ' '
        operon_annot += operon_name
        operons.append(operon_annot)
        for genetic_feat_ID in gen_targs:
            genetic_feat_d = _get_feat_d(RegulonDB_ID=genetic_feat_ID, json=r["genetic features"])
            genetic_feat_annot = ""
            mut_count = genetic_feat_counts[genetic_feat_ID]
            if genetic_feat_d["significant"] and mut_count > 1: genetic_feat_annot += "*"
            genetic_feat_annot += str(mut_count) + ' '
            genetic_feat_annot += str(genetic_feat_d["name"])
            genetic_feats.append(genetic_feat_annot)
            flow_row = {
                "source": operon_annot,
                "target": genetic_feat_annot,
                "type": signif_flow,
                "value": 1
            }
            flow_df = flow_df.append(flow_row, ignore_index=True)
            
            
    if FUNC_ANNOT == "pathway":
        for pathway, ops in r["pathway links"].items():
            pathway_feat_d = _get_feat_d(name=pathway, json=r["pathways"])
            pathway_annot = ""
            mut_count = pathway_counts[pathway]
            if pathway_feat_d["significant"] and mut_count > 1: pathway_annot += "*"
            pathway_annot += str(mut_count) + ' ' + pathway_feat_d["name"]
            pathways.append(pathway_annot)
            for operon_id in ops:
                op_feat_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
                operon_name = operon_df[operon_df["OPERON_ID"]==operon].iloc[0]["OPERON_NAME"]
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": pathway_annot,
                    "target": operon_annot,
                    "type": signif_flow,
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)
            
            
    if FUNC_ANNOT == "regulon":
        for regulator, ops in r["regulator links"].items():
            reg_feat_d = _get_feat_d(name=regulator, json=r["regulators"])
            reg_annot = ""
            mut_count = regulator_counts[regulator]
            if reg_feat_d["significant"] and mut_count > 1: reg_annot += "*"
            reg_annot += str(mut_count) + ' ' + reg_feat_d["name"]
            regulators.append(reg_annot)
            for operon_name in ops:
                op_feat_d = _get_feat_d(name=operon_name, json=r["operons"])
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": reg_annot,
                    "target": operon_annot,
                    "type": signif_flow,
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)

                
    if FUNC_ANNOT == "COG":
        for COG, ops in r["COG links"].items():
#             if COG == "Nucleotide transport and metabolism":
            COG_feat_d = _get_feat_d(name=COG, json=r["COGs"])
            COG_annot = ""
            mut_count = COG_counts[COG]
            if COG_feat_d["significant"] and mut_count > 1: COG_annot += "*"
            COG_annot += str(mut_count) + ' ' + COG_feat_d["name"]
            COGs.append(COG_annot)
            for operon_id in ops:
                op_feat_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
                operon_name = operon_df[operon_df["OPERON_ID"]==operon].iloc[0]["OPERON_NAME"]
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": COG_annot,
                    "target": operon_annot,
                    "type": signif_flow,
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)

    
display(flow_df[:20])

,source,target,type,value
0,*3 thrA,3 thrA SNP,T,1.0
1,*3 thrA,*3 thrA,T,1.0
2,*3 thrLABC,*3 thrA,T,1.0
3,"3 Glycine, serine and threonine metabolism",*3 thrLABC,T,1.0
4,3 Cysteine and methionine metabolism,*3 thrLABC,T,1.0
5,3 Lysine biosynthesis,*3 thrLABC,T,1.0
6,*3 thrA,3 thrA SNP,T,1.0
7,*3 thrA,*3 thrA,T,1.0
8,*3 thrLABC,*3 thrA,T,1.0
9,"3 Glycine, serine and threonine metabolism",*3 thrLABC,T,1.0


In [12]:
# processing to help a cleaner order of features.
# remove duplicates but maintains the order established above
def remove_duplicates(feats):
    l = list()
    for x in feats:
        if x not in l:
            l.append(x)
    return l

mut_feat_effects = remove_duplicates(mut_feat_effects)
genomic_feats = remove_duplicates(genomic_feats)
genetic_feats = remove_duplicates(genetic_feats)
operons = remove_duplicates(operons)
pathways = remove_duplicates(pathways)
regulators = remove_duplicates(regulators)
COGs = remove_duplicates(COGs)

In [14]:
from ipysankeywidget import SankeyWidget
from floweaver import *


nodes = {
    "mutation": ProcessGroup(mut_feat_effects, title="mutation"),
    "genomic feature": ProcessGroup(genomic_feats, title="genomic feature"),
    "genetic feature": ProcessGroup(genetic_feats, title="genetic feature"),
    "operon": ProcessGroup(operons, title="operon"),
}

l = []
if FUNC_ANNOT == "pathway":
    l = pathways
if FUNC_ANNOT == "regulon":
    l = regulators
if FUNC_ANNOT == "COG":
    l = COGs
nodes[FUNC_ANNOT] = ProcessGroup(l, title=FUNC_ANNOT)


# !!! These partition DFs need to have accessible variables (ex: mut_type_partition) accessible external to the nodes
# otherwise floweaver will crash during its rendering step.
mutation_partition = Partition.Simple("process", mut_feat_effects)
nodes["mutation"].partition = mutation_partition

genomic_feat_partition = Partition.Simple("process", genomic_feats)
nodes["genomic feature"].partition = genomic_feat_partition

genetic_feat_partition = Partition.Simple("process", genetic_feats)
nodes["genetic feature"].partition = genetic_feat_partition

operon_partition = Partition.Simple("process", operons)
nodes["operon"].partition = operon_partition

nodes[FUNC_ANNOT].partition = Partition.Simple("process", l)


# The order that the labels is entered into the bundles is explicit. Don't arbitrarily give an order.
bundles = [
    Bundle(
        "genomic feature",
        "mutation"
    ),
    Bundle(
        "genetic feature",
        "genomic feature",
    ),
    Bundle(
        "operon",
        "genetic feature",
    ),
    Bundle(
        FUNC_ANNOT,
        "operon",
    ),
]


ordering = [
    [FUNC_ANNOT],
    ["operon"],
    ["genetic feature"],
    ["genomic feature"],
    ["mutation"]
]


mut_signif_partition = Partition.Simple(
    "type", ['T', 'F'])
sdd = SankeyDefinition(nodes,
                       bundles,
                       ordering,
                       flow_partition=mut_signif_partition
                      )


p_d = {'T': "#6BAAD3", 'F': "#6BAAD3"}
size = dict(width=1200, height=200)
results = weave(sdd,
                flow_df,
                palette=p_d,
                ).to_widget(
    margins=dict(left=400, right=350),
    **size)
results.auto_save_svg("mutflow.svg")
results.auto_save_png("mutflow.png")
results

SankeyWidget(groups=[{'id': 'pathway', 'type': 'process', 'title': 'pathway', 'nodes': ['pathway^3 Glycine, se…